### funcion de agrupacion (group by)
Esta funcion recibe como parametros un dataframe y un diccionario anidado con el nombre de la columna resultado de la agrupacion (como clave) y como valor otro diccionario que tiene como primer elemento la clave tipo y como valor el tipo de agrupacion y como segundo argumento la clave columna y como valor el nombre de la columna que se quiere agrupar , esta funcion retorna un nuevo dataframe con el resultado de la agrupacion.

In [2]:
def group_df(dataframe, aggregation_dictionary, group_by_columns):
    aggregation_expressions = []
    for column_name, aggregation_info in aggregation_dictionary.items():
        aggregation_type = aggregation_info['type']
        column_to_aggregate = aggregation_info['column']
        if aggregation_type == 'count':
            aggregation_expressions.append(F.count(column_to_aggregate).alias(column_name))
        elif aggregation_type == 'sum':
            aggregation_expressions.append(F.sum(column_to_aggregate).alias(column_name))
    grouped_df = dataframe.groupBy(*group_by_columns).agg(*aggregation_expressions)
    return grouped_df

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# SparkSession
spark = SparkSession.builder \
    .appName("Ejemplo PySpark") \
    .getOrCreate()

# DataFrame de ejemplo con las columnas solicitadas
datos = [("1", "Cliente1", 100, 5),
         ("2", "Cliente2", 200, 3),
         ("1", "Cliente1", 300, 7),
         ("2", "Cliente2", 400, 4),
         ("2", "Cliente2", 500, 2)]
df = spark.createDataFrame(datos, ["idcliente", "nombre_cliente", "monto", "cantidadProductos"])

# Mostrar el DataFrame original
print("DataFrame original:")
df.show()

# Definir un diccionario de agrupación de ejemplo
diccionario_agrupacion = {
    "suma_monto": {
        "type": "sum",
        "column": "monto"
    },
    "suma_cantidad_productos": {
        "type": "sum",
        "column": "cantidadProductos"
    }
}
# Probar la función agrupando el DataFrame por la columna segun la lista pasada como parametro
# con el diccionario de agrupación
df_agrupado = group_df(df, diccionario_agrupacion, ["idcliente"])

# Mostrar el DataFrame agrupado
print("DataFrame agrupado por cliente:")
df_agrupado.show()


24/02/23 13:02:36 WARN Utils: Your hostname, merlin-ThinkPad-T470-W10DG resolves to a loopback address: 127.0.1.1; using 192.168.0.9 instead (on interface wlp4s0)
24/02/23 13:02:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/23 13:02:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/23 13:02:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


DataFrame original:


+---------+--------------+-----+-----------------+
|idcliente|nombre_cliente|monto|cantidadProductos|
+---------+--------------+-----+-----------------+
|        1|      Cliente1|  100|                5|
|        2|      Cliente2|  200|                3|
|        1|      Cliente1|  300|                7|
|        2|      Cliente2|  400|                4|
|        2|      Cliente2|  500|                2|
+---------+--------------+-----+-----------------+

DataFrame agrupado por cliente:


+---------+----------+-----------------------+
|idcliente|suma_monto|suma_cantidad_productos|
+---------+----------+-----------------------+
|        1|       400|                     12|
|        2|      1100|                      9|
+---------+----------+-----------------------+



24/02/23 13:02:57 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
